In [ ]:
from tensorflow import keras
keras.__version__

'2.5.0'

In [ ]:
from tensorflow.keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [ ]:
print(train_data.shape, train_targets.shape, test_data.shape, test_targets.shape)

(404, 13) (404,) (102, 13) (102,)


In [ ]:
print(train_data[0], train_targets[0])

[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ] 15.2


 [01]  CRIM 자치시(town) 별 1인당 범죄율  
 [02]  ZN 25,000 평방피트를 초과하는 거주지역의 비율  
 [03]  INDUS 비소매상업지역이 점유하고 있는 토지의 비율  
 [04]  CHAS 찰스강에 대한 더미변수(강의 경계에 위치한 경우는 1, 아니면 0)  
 [05]  NOX 10ppm 당 농축 일산화질소  
 [06]  RM 주택 1가구당 평균 방의 개수  
 [07]  AGE 1940년 이전에 건축된 소유주택의 비율  
 [08]  DIS 5개의 보스턴 직업센터까지의 접근성 지수  
 [09]  RAD 방사형 도로까지의 접근성 지수  
 [10]  TAX 10,000 달러 당 재산세율  
 [11]  PTRATIO 자치시(town)별 학생/교사 비율  
 [12]  B 1000(Bk-0.63)^2, 여기서 Bk는 자치시별 흑인의 비율을 말함.  
 [13]  LSTAT 모집단의 하위계층의 비율(%)  
 [14]  MEDV 본인 소유의 주택가격(중앙값) (단위: $1,000)


In [ ]:
mean = train_data.mean(axis=0)
print(mean.shape)

(13,)


In [ ]:
# train data standardization
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

# test data standardization
test_data -= mean
test_data/= std

# 위 두개는 z - transformation
# 표준화(각 값들의 분산.)

In [ ]:
print(train_data[0], train_targets[0])

[-0.27224633 -0.48361547 -0.43576161 -0.25683275 -0.1652266  -0.1764426
  0.81306188  0.1166983  -0.62624905 -0.59517003  1.14850044  0.44807713
  0.8252202 ] 15.2


In [ ]:
# sigmoid함수 사용한다가정하면
min_of_target = train_targets.min(axis=0)
max_of_target = train_targets.max(axis=0)
print(min_of_target, max_of_target)

5.0 50.0


In [ ]:
normalized_target = (train_targets - min_of_target)/(max_of_target - min_of_target)
print(normalized_target, normalized_target.min(), normalized_target.max())

[0.22666667 0.82888889 1.         0.35777778 0.28222222 0.3
 0.14       0.23555556 0.23555556 0.20888889 0.15777778 0.28666667
 0.40222222 0.33111111 0.23777778 0.08444444 1.         0.38888889
 0.42444444 0.5        0.13111111 0.57333333 0.62       0.42222222
 0.3        0.18444444 0.39777778 0.66       0.25777778 0.27777778
 0.38444444 0.24666667 0.22       0.40222222 0.66444444 0.44444444
 0.19777778 0.18       0.34222222 0.33333333 0.22666667 0.43777778
 0.38222222 0.26       0.17111111 0.23555556 0.29777778 0.35555556
 0.55777778 0.22444444 0.30444444 0.10222222 0.58888889 0.44
 0.31333333 0.37777778 0.21111111 0.13333333 0.6        0.54222222
 0.34       0.43111111 0.21333333 0.32222222 0.20222222 0.20666667
 0.23555556 0.12222222 0.02888889 0.31777778 0.31777778 0.18666667
 0.69777778 0.28444444 0.18888889 0.25555556 0.07333333 0.20666667
 0.24444444 0.18666667 0.52444444 0.85555556 0.33777778 0.37777778
 0.4        0.34888889 0.16666667 0.96666667 0.21333333 0.18666667
 0.41555

In [ ]:
from tensorflow.keras import models
from tensorflow.keras.layers import Dense

In [ ]:
def build_model():
  model = models.Sequential()
  model.add(Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation=None))
  model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
  return model

In [ ]:
import numpy as np

k = 5

num_val_samples = len(train_data) // k
num_epochs = 30
all_scores = []
for i in range(k):
  print('처리중인 폴드 #', i)
  # 검증 데이터 준비: k번째 분할
  val_data = train_data[i * num_val_samples: (i+1)*num_val_samples]
  val_targets = train_targets[i * num_val_samples: (i+1)*num_val_samples]

  # 훈련 데이터 준비: 다른 분할 전체
  partial_train_data = np.concatenate([train_data[:i*num_val_samples],train_data[(i+1)*num_val_samples:]], axis=0)
  partial_train_targets = np.concatenate([train_targets[:i*num_val_samples],train_targets[(i+1)*num_val_samples:]], axis=0)

  # 케라스 모델 구성
  model = build_model()

  # 모델 훈련
  model.fit(partial_train_data, partial_train_targets,
            epochs=num_epochs, batch_size=10,verbose=0)
  
  # 모델 평가
  val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=1)
  print(val_mae)
  all_scores.append(val_mae)

처리중인 폴드 # 0
3/3 [==============================] - 0s 3ms/step - loss: 7.8999 - mae: 1.9807
1.9807170629501343
처리중인 폴드 # 1
3/3 [==============================] - 0s 3ms/step - loss: 13.1806 - mae: 2.8065
2.8064777851104736
처리중인 폴드 # 2
3/3 [==============================] - 0s 3ms/step - loss: 9.8750 - mae: 2.4926
2.4926249980926514
처리중인 폴드 # 3
3/3 [==============================] - 0s 3ms/step - loss: 17.3209 - mae: 2.5342
2.5342049598693848
처리중인 폴드 # 4
3/3 [==============================] - 0s 3ms/step - loss: 16.1244 - mae: 2.8334
2.833406925201416


In [ ]:
all_scores

[1.9807170629501343,
 2.8064777851104736,
 2.4926249980926514,
 2.5342049598693848,
 2.833406925201416]

In [ ]:
np.mean(all_scores)

2.529486346244812

In [ ]:
predictedHousePrice = model.predict(val_data)

In [ ]:
print(predictedHousePrice[1], val_targets[1])

[29.556824] 50.0


In [ ]:
new_data = [1.47, 0.0, 8.14, 0.0, 0.53, 6.1, 91.7, 4.9769,  4.12,  307.6 ,  22,  296.9, 18.72]

In [ ]:
new_data -= mean
new_data /= std

print(new_data.shape)

(13,)


In [ ]:
new_data = np.asarray(new_data)
new_data = np.expand_dims(new_data, axis=0)
print(new_data.shape)

(1, 13)


In [ ]:
model.predict(new_data)

array([[13.980411]], dtype=float32)

In [ ]:
test_mse, test_mae = model.evaluate(test_data, test_targets, verbose=0)
print(test_mse, test_mae)

17.63985824584961 2.89180326461792
